In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
from Bio import SeqIO
from Bio.Align import MultipleSeqAlignment
from collections import Counter
from sklearn.cross_validation import train_test_split, ShuffleSplit
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import auc, roc_auc_score, accuracy_score, roc_curve
from sklearn.preprocessing import LabelBinarizer
from util.isoelectric_point import isoelectric_points

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Problem Statement

There are influenza viruses that are collected from the "environment", or have an "unknown" host. How do we infer which hosts it came from? Well, that sounds like a **Classification** problem. 

In [ ]:
# Load the sequences into memory
sequences = [s for s in SeqIO.parse('data/20160127_HA_prediction.fasta', 'fasta') if len(s.seq) == 566]  # we are cheating and not bothering with an alignment.
len(sequences)

In [ ]:
# Load the sequence IDs into memory
seqids = [s.id for s in SeqIO.parse('data/20160127_HA_prediction.fasta', 'fasta') if len(s.seq) == 566]
len(seqids)

In [ ]:
# Cast the sequences as a MultipleSeqAlignment object, and then turn that into a pandas DataFrame. 
# Note: this cell takes a while.
seq_aln = MultipleSeqAlignment(sequences)
seq_df = pd.DataFrame(np.array(seq_aln))
seq_df.head()

In [ ]:
# Transform the df into isoelectric point features.
seq_feats = seq_df.replace(isoelectric_points.keys(), isoelectric_points.values())
seq_feats.index = seqids
seq_feats.head()

In [ ]:
# Quick check to make sure that we have no strings:
for c in seq_feats.columns:
    letters = set(seq_feats[c])
    for item in letters:
        assert not isinstance(item, str)

In [ ]:
# Let us now load our labels.
labels = pd.read_csv('data/20160127_HA_prediction.csv', parse_dates=['Collection Date'])
labels['Host Species'] = labels['Host Species'].str.replace('IRD:', '').str.replace('/Avian', '')
labels['Sequence Accession'] = labels['Sequence Accession'].str.replace('*', '')
labels.set_index('Sequence Accession', inplace=True)
labels.head()

In [ ]:
# Let's join in the labels so that we have everything in one big massive table.
data_matrix = seq_feats.join(labels['Host Species'], how='inner')
data_matrix.head()

In [ ]:
# Quickly inspect the different labels under "host species"
# set(data_matrix['Host Species'])

In [ ]:
# We will want to predict the labels for: "Avian", "Bird", "Environment", "Unknown", "null"
unknown_labels = ['Avian', 'Bird', 'Environment', 'Unknown', 'null']
known_labels = set(data_matrix['Host Species']) - set(unknown_labels)

In [ ]:
# Let's further split the data into the "unknowns" and the "knowns"
unknowns = data_matrix[data_matrix['Host Species'].isin(unknown_labels)]
knowns = data_matrix[data_matrix['Host Species'].isin(known_labels)]

In [ ]:
# Finally, we want to convert the known host species into a matrix of 1s and 0s, so that we can use them as inputs
# to the training algorithm.

lb = LabelBinarizer()
lb.fit([s for s in known_labels])
lb.transform(knowns['Host Species'])  # note: this has not done anything to the original data.

# Chalk Talk Time!

We're almost ready for training a machine learning model to classify the unknown hosts based on their sequence.

Here's the proper procedure.

1. Split the labelled data into a training and testing set. (~70 train/30 test to 80 train/20 test)
1. Train and evaluate a model on the training set.
1. Make predictions on the testing set, evaluate the model on testing set accuracy.

This procedure is known as cross-validation, and is a powerful, yet cheap & easy method for evaluating how good a particular supervised learning model works.

In [ ]:
# Split the data into a training and testing set.
X_cols = [i for i in range(0,566)]
X = knowns[X_cols]
Y = lb.transform(knowns['Host Species'])

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)

In [ ]:
# Train a Random Forest Classifier.
# Note: This cell takes a while; any questions?

# Initialize the classifier object.
clf = RandomForestClassifier()

# Train (i.e. "fit") the classifier to the training Xs and Ys
clf.fit(X_train, Y_train)

# Make predictions on the test X
preds = clf.predict(X_test)
preds

### How do we evaluate how good the classification task performed?

For binary classification, the Receiver-Operator Characteristic curve is a great way to evaluate a classification task.

For multi-label classification, which is the case we have here, accuracy score is a great starting place.

In [ ]:
# Let's first take a look at the accuracy score: the fraction that were classified correctly.
accuracy_score(lb.inverse_transform(Y_test), lb.inverse_transform(preds))

### What about those sequences for which the hosts were unknown?

We can run the `predict(unknown_Xs)` to predict what their hosts were likely to be, given their sequence.

In [ ]:
unknown_preds = clf.predict(unknowns[X_cols])
unknown_preds = lb.inverse_transform(unknown_preds)
unknown_preds

What this gives us is the class label with the highest probability of being the correct one. 

While we will not do this here, at this point, it would be a good idea to double-check your work with a sanity check. Are the sequences that are predicted to be `Human` truly of a close sequence similarity to actual `Human` sequences? You may want to do a Multiple Sequence Alignment, or you might want to simply compute the Levenshtein or Hamming distance between the two sequences, as a sanity check.

### How do we interpret what the classifier learned?

Depending on the classifier used, you can peer inside the model to get a feel for what the classifier learned about the features that best predict the class label.

The `RandomForestClassifier` provides a `feature_importances_` attribute that we can access and plot.

In [ ]:
plt.plot(clf.feature_importances_)

The interpretation here is that the positions with greater "feature importance" were better at predicting the host class.

## Hands-On Exercises

Now that you've seen the coding pattern for classification using the `RandomForestClassifier`, try it out for the `DecisionTreeClassifier`, evaluate its accuracy, and if applicable, try doing the "peering inside" of the model.

## Conclusions

- Classification: it's all about using features to predict a label.
    - `classifier.fit(train_data)`
    - `classifier.predict(test_data)`
- Split your data into training and testing sets.

# Bonus exercise (time permitting)

Can you train a model to classify the two types of influenza H1N1 sequences from the previous notebook?